In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
import category_encoders as ce   # version 1.2.8
import numpy as np
from sklearn.pipeline import Pipeline

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
import lightgbm as lgb
import tensorflow

/home/vignesh/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/vignesh/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/vignesh/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/vignesh/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:5

# Read data

In [4]:
data = pd.read_csv('../data/carInsurance_train.csv')

In [5]:
from sklearn.model_selection import train_test_split

# Holding out 20% of the sample for test dataset

* Performing stratified sampling
* X_train, y_train - training dataset
* X_test, y_test - test dataset (hold-out)

In [6]:
X = data.drop('CarInsurance', axis=1)
target = data['CarInsurance']
X_train, X_test, y_train, y_test = train_test_split(X, target,  test_size=0.2, random_state=1)

# CallStart and CallEnd is converted to call duration

In [7]:
call_duration_arr = []
for index, row in X_train.iterrows():
    
    call_start = row['CallStart'] 
    call_end = row['CallEnd']
    call_start_hr = call_start.split(':')[0]
    call_start_min = call_start.split(':')[1]
    call_start_sec = call_start.split(':')[2]
    
    call_end_hr = call_end.split(':')[0]
    call_end_min = call_end.split(':')[1]
    call_end_sec = call_end.split(':')[2]
    
    call_start_dt = dt.datetime(2013,12,30,int(call_start_hr),int(call_start_min),int(call_start_sec))
    call_end_dt = dt.datetime(2013,12,30,int(call_end_hr),int(call_end_min),int(call_end_sec))

    call_duration = (call_end_dt-call_start_dt).total_seconds()
    call_duration_arr.append(call_duration)
    
X_train['call_duration'] = call_duration_arr

call_duration_arr = []
for index, row in X_test.iterrows():
    
    call_start = row['CallStart'] 
    call_end = row['CallEnd']
    call_start_hr = call_start.split(':')[0]
    call_start_min = call_start.split(':')[1]
    call_start_sec = call_start.split(':')[2]
    
    call_end_hr = call_end.split(':')[0]
    call_end_min = call_end.split(':')[1]
    call_end_sec = call_end.split(':')[2]
    
    call_start_dt = dt.datetime(2013,12,30,int(call_start_hr),int(call_start_min),int(call_start_sec))
    call_end_dt = dt.datetime(2013,12,30,int(call_end_hr),int(call_end_min),int(call_end_sec))

    call_duration = (call_end_dt-call_start_dt).total_seconds()
    call_duration_arr.append(call_duration)
    
X_test['call_duration'] = call_duration_arr

/home/vignesh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/vignesh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


# Removing unneccessary columns - ID, CallStart, CallEnd, LastContactDay, Default

In [8]:

if len(set(['Id', 'CallStart', 'CallEnd', 'LastContactDay']) - set(list(X_train.columns)))<1:
    X_train = X_train.drop(['Id', 'CallStart', 'CallEnd', 'LastContactDay', 'Default'], axis=1)
    
if len(set(['Id', 'CallStart', 'CallEnd', 'LastContactDay']) - set(list(X_test.columns)))<1:
    X_test = X_test.drop(['Id', 'CallStart', 'CallEnd', 'LastContactDay', 'Default'], axis=1)

In [9]:
X_train.isnull().sum()

Age                    0
Job                   16
Marital                0
Education            130
Balance                0
HHInsurance            0
CarLoan                0
Communication        724
LastContactMonth       0
NoOfContacts           0
DaysPassed             0
PrevAttempts           0
Outcome             2434
call_duration          0
dtype: int64

In [10]:
from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin


# Custom Transformer functions

* Imputation
* Log Transformation
* Normalization

In [11]:
class FillImputer(BaseEstimator, TransformerMixin):
    def __init__(self, fill_dict):
        super().__init__()
        self.fill_dict = fill_dict

    def fit(self, X, y=None):

        return self

    def transform(self, X, y=None):
#         X[:] = (X.to_numpy() - self.means_) / self.std_

        X_copy = X.copy()
        X_copy = X_copy.fillna(self.fill_dict)

        return X_copy
    
class kNNImputer(BaseEstimator, TransformerMixin):
    def __init__(self):
        super().__init__()
        self.kneigh = KNeighborsClassifier(n_neighbors=3)
        self.ce_one_hot = ce.OneHotEncoder(cols = ['Job','Marital'])
        self.ce_one_hot_model = None

    def fit(self, X, y=None):
#         X = X.to_numpy()
#         self.means_ = X.mean(axis=0, keepdims=True)
#         self.std_ = X.std(axis=0, keepdims=True)
        imputation_features_non_missing = X[~X.Education.isnull()][['Job', 'Marital', 'Age']]
        education_non_missing = X[~X.Education.isnull()]['Education']
        
        self.ce_one_hot_model = self.ce_one_hot.fit(imputation_features_non_missing)
        imputation_features_non_missing_encoded = self.ce_one_hot_model.transform(imputation_features_non_missing)
        
        self.kneigh.fit(X=imputation_features_non_missing_encoded, y=education_non_missing)
    
        return self
    

    def transform(self, X, y=None):
#         X[:] = (X.to_numpy() - self.means_) / self.std_
    
#         non_missing = X[~X.Education.isnull()]
#         missing = X[X.Education.isnull()]

        X_copy = X.copy()
    
        missing_ind = X_copy[X_copy.Education.isnull()].index
        education = X_copy.Education.copy()
       
        imputation_features_missing = X_copy[X_copy.Education.isnull()][['Job', 'Marital', 'Age']]
        education_missing = X_copy[X_copy.Education.isnull()]['Education']
                
        imputation_features_missing_encoded = self.ce_one_hot_model.transform(imputation_features_missing)
        missing_values_pred = self.kneigh.predict(imputation_features_missing_encoded)

        counter = 0
        for index in missing_ind:
            education[index] = missing_values_pred[counter]
            counter+=1
        
        X_copy['Education'] = education

        return X_copy

In [12]:
from sklearn.preprocessing import MinMaxScaler as min_max
from sklearn.preprocessing import StandardScaler

class LogTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        super().__init__()
        self.columns = columns

    def fit(self, X, y=None):

        return self

    def transform(self, X, y=None):
        
        X_copy = X.copy()

        for col in self.columns:
            X_copy[col] = np.log(X_copy[col]+1)
        

        return X_copy

class MinMaxScalarMultiple(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        super().__init__()
        self.columns = columns
        self.normalizer = None

    def fit(self, X, y=None):
        self.normalizer = min_max()
        self.normalizer.fit(X[self.columns])

        return self

    def transform(self, X, y=None):
        
        X_copy = X.copy()

        normalized_vals = self.normalizer.transform(X[self.columns])
        
        for i in range(0, len(self.columns)):
            col = [row[i] for row in normalized_vals]
            X_copy[self.columns[i]] = col
        

        return X_copy
    
class StandardScalarMultiple(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        super().__init__()
        self.columns = columns
        self.normalizer = None

    def fit(self, X, y=None):
        self.normalizer = StandardScaler()
        self.normalizer.fit(X[self.columns])

        return self

    def transform(self, X, y=None):
        
        X_copy = X.copy()

        normalized_vals = self.normalizer.transform(X[self.columns])
        
        for i in range(0, len(self.columns)):
            col = [row[i] for row in normalized_vals]
            X_copy[self.columns[i]] = col
        

        return X_copy

In [13]:
def quarter_month(x):
    if x in ['jan', 'feb', 'mar', 'apr']:
        return '1'
    elif x in ['may', 'jun', 'jul', 'aug']:
        return '2'
    elif x in ['sep', 'oct', 'nov', 'dec']:
        return '3'

In [14]:
############### converted negative balance to zero
X_train['Balance'] = X_train.Balance.apply(lambda x : 0 if x<0 else x)

############### create a new variable for indicting whether a client is previously contacted - Boolean
X_train['IsPreviouslyContacted'] = X_train.DaysPassed.apply(lambda x : 0 if x==-1 else 1)

############### convert -ve values for days passed
X_train['DaysPassed'] = X_train.DaysPassed.apply(lambda x : 0 if x==-1 else x)

############### convert month to quadrimester
X_train['LastContactQuadrimester'] = X_train.LastContactMonth.apply(lambda x : quarter_month(x))


# Create a sklearn pipeline for feature engineering - imputation, encoding, scaling

In [15]:
log_transformer = LogTransformer(columns=['call_duration'])
standard_scaler = StandardScalarMultiple(columns=['Age', 'Balance', 'NoOfContacts', 'PrevAttempts', 'DaysPassed'])
fill_imputer = FillImputer(fill_dict={'Communication': 'missing', 'Job' : 'management', 'Outcome' : 'not_contacted'})
knn_imputer = kNNImputer()
one_hot_encoder = ce.OneHotEncoder(cols=['Job', 'Marital', 'Education', 'Outcome', 'Communication', 'LastContactQuadrimester'])


pipe = Pipeline([('log_transformer', log_transformer), 
                 ('standard_scaler', standard_scaler),
                 ('fill_imputer', fill_imputer),
                 ('knn_imputer', knn_imputer),
                 ('one_hot_encoder', one_hot_encoder)])

pipe.fit(X_train)
X_train_features = pipe.transform(X_train)
X_train_features = X_train_features.drop('LastContactMonth', axis=1)

In [16]:
X_train_features[0:10]

,Age,Job_1,Job_2,Job_3,Job_4,Job_5,Job_6,Job_7,Job_8,Job_9,Job_10,Job_11,Marital_1,Marital_2,Marital_3,Education_1,Education_2,Education_3,Balance,HHInsurance,CarLoan,Communication_1,Communication_2,Communication_3,NoOfContacts,DaysPassed,PrevAttempts,Outcome_1,Outcome_2,Outcome_3,Outcome_4,call_duration,IsPreviouslyContacted,LastContactQuadrimester_1,LastContactQuadrimester_2,LastContactQuadrimester_3
2996,-0.367912,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,-0.423515,0,0,1,0,0,-0.196231,-0.464205,-0.373862,1,0,0,0,5.093750,0,1,0,0
3295,1.183915,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0.431397,1,0,1,0,0,-0.521084,-0.464205,-0.373862,1,0,0,0,5.746203,0,1,0,0
2284,-0.023062,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,-0.347487,1,0,0,1,0,-0.521084,-0.464205,-0.373862,1,0,0,0,4.330733,0,1,0,0
2793,-0.367912,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0.553940,0,0,1,0,0,-0.196231,1.424987,2.276469,0,1,0,0,5.676754,1,1,0,0
787,1.528765,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,-0.484145,0,0,1,0,0,1.103179,-0.464205,-0.373862,1,0,0,0,7.050123,0,1,0,0
2046,2.477104,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0.729093,0,0,1,0,0,-0.521084,-0.464205,-0.373862,1,0,0,0,6.428105,0,0,1,0
739,-0.454125,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,-0.484145,1,0,0,1,0,1.752885,-0.464205,-0.373862,1,0,0,0,4.454347,0,1,0,0
2149,0.149363,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,-0.367056,1,0,0,1,0,-0.196231,-0.464205,-0.373862,1,0,0,0,6.008813,0,1,0,0
3369,-0.626550,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0.032973,0,0,1,0,0,6.300822,-0.464205,-0.373862,1,0,0,0,4.691348,0,1,0,0
3492,-0.454125,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,-0.383737,1,1,1,0,0,-0.196231,-0.464205,-0.373862,1,0,0,0,5.579730,0,0,1,0


# Apply pipeline on test dataset

In [17]:
X_test['Balance'] = X_test.Balance.apply(lambda x : 0 if x<0 else x)
X_test['IsPreviouslyContacted'] = X_test.DaysPassed.apply(lambda x : 0 if x==-1 else 1)
X_test['DaysPassed'] = X_test.DaysPassed.apply(lambda x : 0 if x==-1 else x)
X_test['LastContactQuadrimester'] = X_test.LastContactMonth.apply(lambda x : quarter_month(x))

# X_test_1 = log_transformer.transform(X_test)
# X_test_2 = standard_scaler.transform(X_test_1)
# X_test_3 = fill_imputer.transform(X_test_2)
# X_test_4 = knn_imputer.transform(X_test_3)
# X_test_5 = one_hot_encoder.transform(X_test_4)

X_test_features = pipe.transform(X_test)
X_test_features = X_test_features.drop('LastContactMonth', axis=1)

In [18]:
X_test_features[0:5]

,Age,Job_1,Job_2,Job_3,Job_4,Job_5,Job_6,Job_7,Job_8,Job_9,Job_10,Job_11,Marital_1,Marital_2,Marital_3,Education_1,Education_2,Education_3,Balance,HHInsurance,CarLoan,Communication_1,Communication_2,Communication_3,NoOfContacts,DaysPassed,PrevAttempts,Outcome_1,Outcome_2,Outcome_3,Outcome_4,call_duration,IsPreviouslyContacted,LastContactQuadrimester_1,LastContactQuadrimester_2,LastContactQuadrimester_3
200,1.270127,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,2.134486,0,0,1,0,0,0.453474,-0.464205,-0.373862,1,0,0,0,7.116394,0,1,0,0
1078,-0.798975,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,-0.416779,0,0,1,0,0,0.128621,-0.464205,-0.373862,1,0,0,0,4.804021,0,1,0,0
610,1.528765,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,-0.136085,0,1,1,0,0,0.453474,-0.464205,-0.373862,1,0,0,0,4.624973,0,1,0,0
2159,0.149363,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,-0.227511,1,0,1,0,0,-0.521084,-0.464205,-0.373862,1,0,0,0,6.805723,0,1,0,0
1169,1.270127,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,-0.133519,0,0,1,0,0,0.453474,-0.464205,-0.373862,1,0,0,0,4.499810,0,1,0,0


# Fit the logistic regression model on train dataset and predict on test dataset

In [26]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [27]:
lg = LogisticRegression(max_iter=500)
lg.fit(X_train_features, y_train)

LogisticRegression(max_iter=500)

In [30]:
from sklearn.model_selection import GridSearchCV
param_grid = {
   'C': [0.001, 0.01, 0.1, 1]
    }
kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=7)

search = GridSearchCV(lg, param_grid, cv=kfold, scoring='accuracy', verbose=1, n_jobs=8).fit(X_train_features, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  10 out of  12 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  12 out of  12 | elapsed:    0.2s finished


In [31]:
lg = search.best_estimator_
lg

LogisticRegression(C=1, max_iter=500)

In [32]:
cross_val_score(LogisticRegression(C=1, max_iter=500), X_train_features, y_train, cv=kfold, scoring='accuracy')

array([0.81818182, 0.81818182, 0.8011257 ])

In [35]:
np.mean(cross_val_score(LogisticRegression(C=1, max_iter=500), X_train_features, y_train, cv=kfold, scoring='accuracy'))

0.8124964466427881

In [33]:
cross_val_score(LogisticRegression(C=1, max_iter=500), X_train_features, y_train, cv=kfold, scoring='precision')

array([0.79259259, 0.77116705, 0.75348837])

In [36]:
np.mean(cross_val_score(LogisticRegression(C=1, max_iter=500), X_train_features, y_train, cv=kfold, scoring='precision'))

0.7724160042468453

In [34]:
cross_val_score(LogisticRegression(C=1, max_iter=500), X_train_features, y_train, cv=kfold, scoring='recall')

array([0.74477958, 0.78190255, 0.75348837])

In [37]:
np.mean(cross_val_score(LogisticRegression(C=1, max_iter=500), X_train_features, y_train, cv=kfold, scoring='recall'))

0.7600568355545964

# Fit the logistic regression model on train dataset and evaluate on test dataset

In [136]:
lg.fit(X_train_features, y_train)

LogisticRegression(C=1, max_iter=500)

In [137]:
from sklearn.metrics import *

prob = lg.predict_proba(X_test_features)
pos_prob = [x[1] for x in prob]
fpr, tpr, thresholds = roc_curve(y_test, pos_prob, pos_label=1.0)
auc(fpr, tpr)

0.8897777427490542

In [42]:
lg_pred = lg.predict(X_test_features)

In [43]:
accuracy_score(y_test, lg_pred)

0.80125

In [44]:
precision_score(y_test, lg_pred)

0.744408945686901

In [45]:
recall_score(y_test, lg_pred)

0.7467948717948718

# Create the random forest model on train dataset and perform cross validation

In [46]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

In [47]:
rf = RandomForestClassifier()

In [135]:
from sklearn.model_selection import GridSearchCV
param_grid = {
   'n_estimators': [5, 10, 20, 30, 50],
   'max_depth': [5, 10, 15, 20],
   'min_samples_split': [20, 30, 50, 70, 100]
    }
search = GridSearchCV(rf, param_grid, cv=kfold, scoring='accuracy', verbose=1, n_jobs=8).fit(X_train_features, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    2.0s
[Parallel(n_jobs=8)]: Done 272 tasks      | elapsed:    5.5s
[Parallel(n_jobs=8)]: Done 285 out of 300 | elapsed:    5.7s remaining:    0.3s
[Parallel(n_jobs=8)]: Done 300 out of 300 | elapsed:    5.9s finished


In [50]:
search.best_params_

{'max_depth': 15, 'min_samples_split': 70, 'n_estimators': 20}

In [49]:
rf = search.best_estimator_
rf

RandomForestClassifier(max_depth=15, min_samples_split=70, n_estimators=20)

In [63]:
cross_val_score(rf, X_train_features, y_train, cv=kfold, scoring='accuracy')

array([0.83786317, 0.83411434, 0.81707317])

In [64]:
np.mean(cross_val_score(rf, X_train_features, y_train, cv=kfold, scoring='accuracy'))

0.8224968393437089

In [65]:
cross_val_score(rf, X_train_features, y_train, cv=kfold, scoring='precision')

array([0.80339806, 0.77951002, 0.76869159])

In [66]:
np.mean(cross_val_score(rf, X_train_features, y_train, cv=kfold, scoring='precision'))

0.782464398355943

In [67]:
cross_val_score(rf, X_train_features, y_train, cv=kfold, scoring='recall')

array([0.7424594 , 0.82598608, 0.80697674])

In [68]:
np.mean(cross_val_score(rf, X_train_features, y_train, cv=kfold, scoring='recall'))

0.7755499199625892

In [51]:
# rf = RandomForestClassifier(n_estimators=50, max_depth=5)
rf.fit(X_train_features, y_train)

RandomForestClassifier(max_depth=15, min_samples_split=70, n_estimators=20)

In [52]:
rf.feature_importances_

array([0.04082135, 0.00278027, 0.00607724, 0.00265355, 0.00167059,
       0.00482745, 0.00195389, 0.00176331, 0.00272247, 0.00116828,
       0.0083473 , 0.00408984, 0.00858833, 0.00876867, 0.00329316,
       0.00114971, 0.00488337, 0.00358714, 0.03022218, 0.0412542 ,
       0.0051824 , 0.01887828, 0.04173484, 0.00085958, 0.01342354,
       0.04188676, 0.03077121, 0.00657229, 0.00280797, 0.08306979,
       0.00931906, 0.51306497, 0.00122127, 0.03316494, 0.00753647,
       0.00988432])

In [53]:
feature_imp_arr = pd.DataFrame({'columns': X_train_features.columns, 'feature_importances': rf.feature_importances_})

In [54]:
# feature_imp_arr.sort_values(by='feature_importances', ascending=False)

# Fit the random forest model on train dataset and evaluate on test dataset

In [59]:
rf_pred = rf.predict(X_test_features)

In [60]:
accuracy_score(y_test, rf_pred)

0.81125

In [61]:
precision_score(y_test, rf_pred)

0.7571884984025559

In [62]:
recall_score(y_test, rf_pred)

0.7596153846153846

In [128]:
from sklearn.metrics import *

prob = rf.predict_proba(X_test_features)
pos_prob = [x[1] for x in prob]
fpr, tpr, thresholds = roc_curve(y_test, pos_prob, pos_label=1.0)
auc(fpr, tpr)

0.9059971364018495

# Fit the gradient boosting model on train dataset and perform cross validation

In [69]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score

In [70]:
gd = GradientBoostingClassifier()

In [75]:
from sklearn.model_selection import GridSearchCV
param_grid = {
   'n_estimators': [5, 10, 20, 30, 50],
   'max_depth': [5, 10, 15, 20],
   'min_samples_split': [20, 30, 50, 70, 100],
   'learning_rate' : [0.1, 0.001, 0.01]
    }
search = GridSearchCV(gd, param_grid, cv=kfold, scoring='accuracy', verbose=1, n_jobs=8).fit(X_train_features, y_train)

Fitting 3 folds for each of 300 candidates, totalling 900 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  52 tasks      | elapsed:    1.4s
[Parallel(n_jobs=8)]: Done 240 tasks      | elapsed:   21.0s
[Parallel(n_jobs=8)]: Done 490 tasks      | elapsed:   44.9s
[Parallel(n_jobs=8)]: Done 840 tasks      | elapsed:  1.3min
[Parallel(n_jobs=8)]: Done 900 out of 900 | elapsed:  1.4min finished


In [77]:
search.best_params_

{'learning_rate': 0.1,
 'max_depth': 20,
 'min_samples_split': 100,
 'n_estimators': 50}

In [82]:
gb = GradientBoostingClassifier(max_depth=20, min_samples_split=100, n_estimators=50, learning_rate=0.1)

In [92]:
cross_val_score(rf, X_train_features, y_train, cv=kfold, scoring='accuracy')

array([0.79475164, 0.83598875, 0.81894934])

In [93]:
np.mean(cross_val_score(rf, X_train_features, y_train, cv=kfold, scoring='accuracy'))

0.8243733050119775

In [96]:
cross_val_score(rf, X_train_features, y_train, cv=kfold, scoring='precision')

array([0.79310345, 0.78426966, 0.75213675])

In [94]:
np.mean(cross_val_score(rf, X_train_features, y_train, cv=kfold, scoring='precision'))

0.7798848474222076

In [97]:
cross_val_score(rf, X_train_features, y_train, cv=kfold, scoring='recall')

array([0.77030162, 0.82134571, 0.80697674])

In [95]:
np.mean(cross_val_score(rf, X_train_features, y_train, cv=kfold, scoring='recall'))

0.7825248655551359

# Fit the gradient boosting model on train dataset and evaluate on test dataset

In [83]:
gb.fit(X_train_features, y_train)

GradientBoostingClassifier(max_depth=20, min_samples_split=100, n_estimators=50)

In [88]:
gb_pred = gb.predict(X_test_features)

In [89]:
accuracy_score(y_test, gb_pred)

0.84

In [90]:
precision_score(y_test, gb_pred)

0.7771084337349398

In [91]:
recall_score(y_test, gb_pred)

0.8269230769230769

In [138]:
from sklearn.metrics import *

prob = gb.predict_proba(X_test_features)
pos_prob = [x[1] for x in prob]
fpr, tpr, thresholds = roc_curve(y_test, pos_prob, pos_label=1.0)
auc(fpr, tpr)

0.9090019440941572

# Create xg boost model and perform cross validation

In [98]:
import xgboost as xgb
xg = xgb.XGBClassifier(n_estimators=50, learning_rate=0.1, max_depth=5, )

In [99]:
from sklearn.model_selection import GridSearchCV
param_grid = {
   'n_estimators': [5, 10, 20, 30, 50],
   'max_depth': [5, 10, 15, 20],
   'learning_rate': [0.001, 0.01, 0.1]
    }
search = GridSearchCV(xg, param_grid, cv=kfold, scoring='accuracy', verbose=1, n_jobs=-1).fit(X_train_features, y_train)

Fitting 3 folds for each of 60 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:    8.0s finished


In [104]:
search.best_params_

{'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 30}

In [139]:
xg = xgb.XGBClassifier(learning_rate=0.1, max_depth= 5, n_estimators=30)

In [106]:
cross_val_score(xg, X_train_features, y_train, cv=kfold, scoring='accuracy')

array([0.83786317, 0.83223993, 0.81613508])

In [107]:
np.mean(cross_val_score(xg, X_train_features, y_train, cv=kfold, scoring='accuracy'))

0.8287460590704242

In [108]:
cross_val_score(xg, X_train_features, y_train, cv=kfold, scoring='precision')

array([0.79587156, 0.77272727, 0.74579832])

In [109]:
np.mean(cross_val_score(xg, X_train_features, y_train, cv=kfold, scoring='precision'))

0.7714657172293439

In [110]:
cross_val_score(xg, X_train_features, y_train, cv=kfold, scoring='recall')

array([0.80510441, 0.82830626, 0.8255814 ])

In [112]:
np.mean(cross_val_score(xg, X_train_features, y_train, cv=kfold, scoring='recall'))

0.8196640227342219

# Fit the xg boost model on train dataset and evaluate

In [114]:
xg.fit(X_train_features, y_train)

XGBClassifier(max_depth=5, n_estimators=30)

In [117]:
xgb_pred = xg.predict(X_test_features)

In [118]:
accuracy_score(y_test, xgb_pred)

0.82875

In [121]:
precision_score(y_test, xgb_pred)

0.7565982404692082

In [122]:
recall_score(y_test, xgb_pred)

0.8269230769230769

In [132]:
from sklearn.metrics import *

prob = xg.predict_proba(X_test_features)
pos_prob = [x[1] for x in prob]
fpr, tpr, thresholds = roc_curve(y_test, pos_prob, pos_label=1.0)
auc(fpr, tpr)

0.9094945355191257